# Deep Learning Tutorial 2: Preprocessing

In this notebook, we will convert our data into a time series for each drive.

In [1]:
from __future__ import division
import os
from glob import glob

import pandas as pd
pd.options.display.max_columns = 1000
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import numpy as np

### Let's have a look at the first ten rows of our data

In [2]:
pd.read_csv('data_Q3_2016/2016-07-01.csv', nrows=10)

,date,serial_number,model,capacity_bytes,failure,smart_1_normalized,smart_1_raw,smart_2_normalized,smart_2_raw,smart_3_normalized,smart_3_raw,smart_4_normalized,smart_4_raw,smart_5_normalized,smart_5_raw,smart_7_normalized,smart_7_raw,smart_8_normalized,smart_8_raw,smart_9_normalized,smart_9_raw,smart_10_normalized,smart_10_raw,smart_11_normalized,smart_11_raw,smart_12_normalized,smart_12_raw,smart_13_normalized,smart_13_raw,smart_15_normalized,smart_15_raw,smart_22_normalized,smart_22_raw,smart_183_normalized,smart_183_raw,smart_184_normalized,smart_184_raw,smart_187_normalized,smart_187_raw,smart_188_normalized,smart_188_raw,smart_189_normalized,smart_189_raw,smart_190_normalized,smart_190_raw,smart_191_normalized,smart_191_raw,smart_192_normalized,smart_192_raw,smart_193_normalized,smart_193_raw,smart_194_normalized,smart_194_raw,smart_195_normalized,smart_195_raw,smart_196_normalized,smart_196_raw,smart_197_normalized,smart_197_raw,smart_198_normalized,smart_198_raw,smart_199_normalized,smart_199_raw,smart_200_normalized,smart_200_raw,smart_201_normalized,smart_201_raw,smart_220_normalized,smart_220_raw,smart_222_normalized,smart_222_raw,smart_223_normalized,smart_223_raw,smart_224_normalized,smart_224_raw,smart_225_normalized,smart_225_raw,smart_226_normalized,smart_226_raw,smart_240_normalized,smart_240_raw,smart_241_normalized,smart_241_raw,smart_242_normalized,smart_242_raw,smart_250_normalized,smart_250_raw,smart_251_normalized,smart_251_raw,smart_252_normalized,smart_252_raw,smart_254_normalized,smart_254_raw,smart_255_normalized,smart_255_raw
0,2016-07-01,MJ0351YNG9Z0XA,Hitachi HDS5C3030ALA630,3000592982016,0,100,0,135.0,108.0,127,554,100,15,100,0,100,0,122.0,37.0,96,32296,100,0,NaN,NaN,100,15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100,319,100,319,253,23,NaN,NaN,100.0,0.0,100,0,100,0,200,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2016-07-01,MJ0351YNG9WJSA,Hitachi HDS5C3030ALA630,3000592982016,0,100,0,136.0,104.0,126,555,100,20,100,2,100,0,122.0,37.0,96,32363,100,0,NaN,NaN,100,20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100,358,100,358,206,29,NaN,NaN,100.0,2.0,100,0,100,0,200,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2016-07-01,PL1321LAG34XWH,Hitachi HDS5C4040ALE630,4000787030016,0,100,0,134.0,101.0,130,560,100,30,100,0,100,0,111.0,43.0,96,28956,100,0,NaN,NaN,100,30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100,41,100,41,200,30,NaN,NaN,100.0,0.0,100,0,100,0,200,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2016-07-01,MJ0351YNGABYAA,Hitachi HDS5C3030ALA630,3000592982016,0,100,0,136.0,104.0,137,507,100,16,100,0,100,0,122.0,37.0,96,31006,100,0,NaN,NaN,100,16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100,339,100,339,200,30,NaN,NaN,100.0,0.0,100,0,100,0,200,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2016-07-01,Z305B2QN,ST4000DM000,4000787030016,0,117,144377848,NaN,NaN,95,0,100,5,100,0,83,202804653,NaN,NaN,95,4778,100,0,NaN,NaN,100,5,NaN,NaN,NaN,NaN,NaN,NaN,100.0,0.0,100.0,0.0,100.0,0.0,100.0,0.0,100.0,0.0,77.0,23.0,100.0,0.0,100,0,84,33793,23,23,NaN,NaN,NaN,NaN,100,0,100,0,200,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.0,4531.0,100.0,2.230474e+10,100.0,2.562937e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2016-07-01,PL2331LAGN2YTJ,HGST HMS5C4040BLE640,4000787030016,0,100,0,133.0,104.0,100,0,100,4,100,0,100,0,111.0,43.0,98,18803,100,0,NaN,NaN,100,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100,157,100,157,250,24,NaN,NaN,100.0,0.0,100,0,100,0,200,0,NaN,NaN,Na

Each drive has a serial number, a model name, and a bunch of S.M.A.R.T. attributes. As far as S.M.A.R.T is concerned, have a look at [the Wikipedia page](https://en.wikipedia.org/wiki/S.M.A.R.T.#ATA_S.M.A.R.T._attributes).

### Feature Selection

For the sake of simplicitly, the aim of this workshop, and limited RAM, we will only consider a few smart attributes which are [known to be predictive](https://www.backblaze.com/blog/what-smart-stats-indicate-hard-drive-failures/).

In [3]:
columns = [
    'date',
    'serial_number',
    'model',
    'capacity_bytes',
    'smart_1_raw',
    'smart_3_raw',
    'smart_4_raw',
    'smart_5_raw',
    'smart_7_raw',
    'smart_9_raw',
    'smart_10_raw',
    'smart_12_raw',
    'smart_187_raw',
    'smart_188_raw',
    'smart_194_raw',
    'smart_197_raw',
    'smart_198_raw',
    'smart_199_raw',
    'failure']

### Read all of our data

In [4]:
data = pd.concat((pd.read_csv(f, usecols=columns + ['model'], parse_dates=['date'])
                  for f in glob("data_Q3_2016/*.csv")), ignore_index=True)

### Normalization

In neural networks, it is advisable to scale attributes. Here, we do it by subtracting the mean and dividing by the standard deviation.

In [5]:
for c in columns[4:-1]:
    data[[c]] = data[[c]].fillna(0)
    data[[c]] = StandardScaler().fit_transform(data[[c]].as_matrix())

/home/fabian/.local/lib/python2.7/site-packages/sklearn/utils/validation.py:429: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, _DataConversionWarning)


### Data Grouping

This is often the most cumbersome step in preprocessing data for use in recurrent neural networks. A recurrent neural network summarizes variable-length sequences into fixed-size vectors. In our case, we create a sequence for each drive in our data set, and each drive is identified by its serial number.

In the following cells, we will create a time series for each drive. In practice, multi-dimensional arrays do not support variable length axes. Hence, if we don't have enough observation for one drive, we need to insert a placeholder in either the beginning or the end of the sequence. For our drives, the lack of observations is most likely due to the drive being added recently, in which case it makes more sense to insert these placeholders in the beginning of the time series.

In [6]:
data.sort_values(by=['serial_number', 'date'], inplace=True)

In [7]:
grouped = data.drop('date', 1).groupby('serial_number', sort=False)

We encode the drive series using one-hot encoding. This helps the model learn about failures rates of drive series instead of treating each drive independently.

In [8]:
models = pd.get_dummies(grouped.first()["model"]).as_matrix()

Let's have a look at the first record for the first drive in our grouping so that we can infer the number of features our final data set will have.

In [9]:
for name, first in grouped:
    first = first.drop('serial_number', 1).drop('model', 1).drop('failure', 1).as_matrix()
    break

In [10]:
max_records = 90

In [11]:
mat = np.zeros((len(grouped), max_records, first.shape[1]))
labels = np.zeros((len(grouped), 1))

In [12]:
mat.shape

(74574, 90, 15)

That looks about right. We have 74574 drives, 100 observations per drive, and 78 variables for each observation.

In the next step, we will fill our final array with our time series.

In [13]:
for i, (name, records) in enumerate(grouped):
    # If the drive failed and was removed, it has a 1 in the last observation.
    labels[i] = [1 if records['failure'].sum() > 0 else 0]
    
    # We select the last max_records observations (or fewer in case there aren't enough)
    row = records.drop('serial_number', 1).drop('model', 1).drop('failure', 1).as_matrix()[-max_records:,]
    
    # We fill up our time series to max_records observations by adding zero vectors in front.
    missing = max_records - row.shape[0]
    if missing > 0:
        row = np.pad(row, ((missing, 0), (0, 0)), 'constant', constant_values=0)
        
    # Finally, we assign our (100, 78) matrix to mat[i]
    mat[i] = row

In [14]:
del data

In [15]:
np.savez("data.npz", records=mat, labels=labels, models=models)